In [3]:
# Import packages
import datetime
import math
import matplotlib.pyplot as plt 
import numpy as np
import pandas as pd

from datetime import date

## Load Datasets

(1) URA Dataset

In [4]:
ura_url = 'https://raw.githubusercontent.com/nicolepng/BT4222/main/Data/ura_5y_trans.csv'
ura_5y_trans = pd.read_csv(ura_url)
ura_5y_trans.drop(ura_5y_trans.columns[0], axis=1, inplace=True)
print(ura_5y_trans.shape)
ura_5y_trans.head()

(73838, 19)


,street,project,marketSegment,area,floorRange,noOfUnits,contractDate,typeOfSale,price,propertyType,district,typeOfArea,tenure,x,y,nettPrice,latitude,longitude,year
0,ZEHNDER ROAD,LANDED HOUSING DEVELOPMENT,RCR,524.3,~-,1,220,3,5500000.0,Semi-detached,5,Land,Freehold,NaN,NaN,NaN,1.282130,103.786879,20
1,ZEHNDER ROAD,LANDED HOUSING DEVELOPMENT,RCR,308.0,~-,1,918,3,5000000.0,Semi-detached,5,Land,Freehold,NaN,NaN,NaN,1.282130,103.786879,18
2,ZEHNDER ROAD,LANDED HOUSING DEVELOPMENT,RCR,314.0,~-,1,618,3,4750000.0,Semi-detached,5,Land,Freehold,NaN,NaN,NaN,1.282130,103.786879,18
3,NEO PEE TECK LANE,LANDED HOUSING DEVELOPMENT,RCR,159.3,~-,1,320,3,2630000.0,Terrace,5,Land,Freehold,NaN,NaN,NaN,1.292047,103.768591,20
4,COVE DRIVE,TURQUOISE,CCR,223.0,~01-05,1,221,3,2800000.0,Condominium,4,Strata,99 yrs lease commencing from 2007,28382.47067,25008.33592,NaN,1.244208,103.827487,21


(2) Ameneties Dataset

In [5]:
amenities_url = 'https://raw.githubusercontent.com/nicolepng/BT4222/main/Data/ameneties_per_district.csv' 
ameneties_per_district = pd.read_csv(amenities_url)
ameneties_per_district.drop(ameneties_per_district.columns[0], axis=1, inplace=True)
print(ameneties_per_district.shape)
ameneties_per_district.head()

(28, 4)


,district,school,hawkercentre,supermarkets
0,1,"['DUNMAN HIGH SCHOOL', 'QUEENSTOWN SECONDARY S...","['Market Street Interim Hawker Centre', 'Maxwe...",[]
1,2,"['DUNMAN HIGH SCHOOL', 'QUEENSTOWN SECONDARY S...",['Tanjong Pagar Plaza Blk 6 (Blk 6 Tanjong Pag...,[]
2,3,"['QUEENSTOWN SECONDARY SCHOOL', 'SCHOOL OF SCI...",['Bukit Merah View Blk 115 (Blk 115 Bukit Mera...,"['FISHOP PTE. LTD.', 'NTUC FAIRPRICE CO-OPERAT..."
3,4,"['FAIRFIELD METHODIST SCHOOL (SECONDARY)', 'QU...",['Telok Blangah Drive Blk 82 (Telok Blangah Ma...,"['NTUC FAIRPRICE CO-OPERATIVE LTD', 'THE QUALI..."
4,5,"['HILLGROVE SECONDARY SCHOOL', 'QUEENSTOWN SEC...","['Pasir Panjang Food Centre', 'Clementi West S...","['NTUC FAIRPRICE CO-OPERATIVE LTD', 'COLD STOR..."


(3) Bus Stops Dataset

In [6]:
# Obtain bus stop data
bus_stops_url = 'https://raw.githubusercontent.com/nicolepng/BT4222/main/Data/bus_stops.csv' 
bus_stops = pd.read_csv(bus_stops_url)
bus_stops.drop(bus_stops.columns[0], axis=1, inplace=True)
print(bus_stops.shape)
bus_stops.head()

(5021, 5)


,BusStopCode,RoadName,Description,Latitude,Longitude
0,481,Woodlands Rd,BT PANJANG TEMP BUS PK,1.383764,103.758300
1,1012,Victoria St,Hotel Grand Pacific,1.296848,103.852536
2,1013,Victoria St,St. Joseph's Ch,1.297710,103.853225
3,1019,Victoria St,Bras Basah Cplx,1.296990,103.853022
4,1029,Nth Bridge Rd,Cosmic Insurance Bldg,1.296673,103.854414


(4.1) Crime Locations Dataset - Overview

In [7]:
crime_locations_url = 'https://raw.githubusercontent.com/nicolepng/BT4222/main/Data/overview_years_crimes_locations_v3.csv' 
overview_years_crimes_locations_v3 = pd.read_csv(crime_locations_url)
overview_years_crimes_locations_v3.drop(overview_years_crimes_locations_v3.columns[3], axis=1, inplace=True) # dropped 'Number (Cases)' column coz only difference between this and 'Number' is that 'Number' states the 0s
print(overview_years_crimes_locations_v3.shape)
overview_years_crimes_locations_v3.head()

(1680, 6)


,Year,NPC,Offence,Neighbourhood,Postal,Number
0,2012,Central Police Division - Total,Robbery,[Total]Central Police Division - Total,-,34
1,2012,Central Police Division - Total,Housebreaking,[Total]Central Police Division - Total,-,32
2,2012,Central Police Division - Total,Snatch Theft,[Total]Central Police Division - Total,-,31
3,2012,Central Police Division - Total,Theft Of Motor Vehicle,[Total]Central Police Division - Total,-,47
4,2012,Central Police Division - Total,Outrage Of Modesty,[Total]Central Police Division - Total,-,193


(4.2) Crime Locations Dataset - Average

In [8]:
avg_crime_locations_url = 'https://raw.githubusercontent.com/nicolepng/BT4222/main/Data/average_crimes_by_location_v3.csv' 
average_crimes_by_location_v3 = pd.read_csv(avg_crime_locations_url)
average_crimes_by_location_v3

,Neighbourhood,Postal,Offence,Number
0,Ang Mo Kio North,569784,Housebreaking,4.857143
1,Ang Mo Kio North,569784,Outrage Of Modesty,26.000000
2,Ang Mo Kio North,569784,Robbery,2.000000
3,Ang Mo Kio North,569784,Snatch Theft,2.714286
4,Ang Mo Kio North,569784,Theft Of Motor Vehicle,5.000000
...,...,...,...,...
210,[Total]Woodlands Police Division - Total,-,Housebreaking,0.285714
211,[Total]Woodlands Police Division - Total,-,Outrage Of Modesty,2.142857
212,[Total]Woodlands Police Division - Total,-,Robbery,0.000000
213,[Total]Woodlands Police Division - Total,-,Snatch Theft,0.000000


(5) Resale Flat Prices Dataset

In [9]:
resale_flat_prices_url = 'https://raw.githubusercontent.com/nicolepng/BT4222/main/Data/resale_flat_prices_based_on_registration_date_from_jan_2017_onwards.csv'
resale_flat_prices = pd.read_csv(resale_flat_prices_url)
resale_flat_prices.drop(resale_flat_prices.columns[0], axis=1, inplace=True)
print(resale_flat_prices.shape)
resale_flat_prices.head()

(92235, 14)


,month,town,flat_type,block,street_name,storey_range,floor_area_sqm,flat_model,lease_commence_date,remaining_lease,resale_price,street,latitude,longitude
0,2017-01,ANG MO KIO,2 ROOM,406,ANG MO KIO AVE 10,10 TO 12,44.0,Improved,1979,61 years 04 months,232000.0,ANG MO KIO AVE 10,1.367602,103.856844
1,2017-01,ANG MO KIO,3 ROOM,108,ANG MO KIO AVE 4,01 TO 03,67.0,New Generation,1978,60 years 07 months,250000.0,ANG MO KIO AVE 4,1.382319,103.838706
2,2017-01,ANG MO KIO,3 ROOM,602,ANG MO KIO AVE 5,01 TO 03,67.0,New Generation,1980,62 years 05 months,262000.0,ANG MO KIO AVE 5,1.376825,103.847556
3,2017-01,ANG MO KIO,3 ROOM,465,ANG MO KIO AVE 10,04 TO 06,68.0,New Generation,1980,62 years 01 month,265000.0,ANG MO KIO AVE 10,1.367602,103.856844
4,2017-01,ANG MO KIO,3 ROOM,601,ANG MO KIO AVE 5,01 TO 03,67.0,New Generation,1980,62 years 05 months,265000.0,ANG MO KIO AVE 5,1.376825,103.847556


In [10]:
sentiment_url = 'https://raw.githubusercontent.com/nicolepng/BT4222/main/Data/combined_sentiment.csv' 
sentiment = pd.read_csv(sentiment_url)
sentiment

,district_num,hwz_sentiment,hwz_comment_count,year,reddit_sentiment,reddit_comment_count,total_count,weighted_sentiment
0,21,0.000000,0,2018,0.145982,14,14,0.145982
1,1,0.082516,8,2018,0.079938,676,684,0.079968
2,2,0.317045,1,2018,0.087518,23,24,0.097082
3,3,0.123678,39,2018,0.055071,571,610,0.059457
4,4,0.093259,30,2018,0.051841,813,843,0.053315
...,...,...,...,...,...,...,...,...
107,24,0.150795,21,2021,0.077776,642,663,0.080089
108,25,0.088860,12,2021,0.043112,239,251,0.045299
109,26,-0.025054,1,2021,0.000000,0,1,-0.025054
110,27,0.084183,45,2021,0.031414,319,364,0.037938


## 2 BIG DATA SETS

## 1. URA
    - ameneties_in_district = ura_5y_trans + ameneties_per_district
    - ameneties_in_district = ameneties_in_district + bus_stops
    - ameneties_and_crimes_in_district = ameneties_in_district + average_crimes_by_location_v3
            
## 2. RESALE FLAT PRICES
    - resale_flats_in_district = resale_flat_prices + ameneties_per_district
    - resale_flats_in_district = resale_flats_in_district + bus_stops 
    - resale_flats_and_crime_in_district = resale_flats_in_district + average_crimes_by_location_v3
            
## Append resale_flats_and_crime_in_district to ameneties_and_crimes_in_district
    1. district
    2. street
    3. propertyType (add [propertyType]=Flat in resale_flat_prices)
    4. remaining_lease 
    5. price (rename resale_flat_prices[resale_price] as [price]])
    6. school
    7. hawkercentre
    8. supermarkets
    9. Bus Stops Nearby
    10. Crime Number

## 1.1 Combine URA and ameneties

In [11]:
# combine ura_5y_trans and ameneties_per_district
ameneties_in_district = ura_5y_trans.join(ameneties_per_district.set_index('district'), on='district')
print(ameneties_in_district.shape)
ameneties_in_district.head()

(73838, 22)


,street,project,marketSegment,area,floorRange,noOfUnits,contractDate,typeOfSale,price,propertyType,...,tenure,x,y,nettPrice,latitude,longitude,year,school,hawkercentre,supermarkets
0,ZEHNDER ROAD,LANDED HOUSING DEVELOPMENT,RCR,524.3,~-,1,220,3,5500000.0,Semi-detached,...,Freehold,NaN,NaN,NaN,1.282130,103.786879,20,"['HILLGROVE SECONDARY SCHOOL', 'QUEENSTOWN SEC...","['Pasir Panjang Food Centre', 'Clementi West S...","['NTUC FAIRPRICE CO-OPERATIVE LTD', 'COLD STOR..."
1,ZEHNDER ROAD,LANDED HOUSING DEVELOPMENT,RCR,308.0,~-,1,918,3,5000000.0,Semi-detached,...,Freehold,NaN,NaN,NaN,1.282130,103.786879,18,"['HILLGROVE SECONDARY SCHOOL', 'QUEENSTOWN SEC...","['Pasir Panjang Food Centre', 'Clementi West S...","['NTUC FAIRPRICE CO-OPERATIVE LTD', 'COLD STOR..."
2,ZEHNDER ROAD,LANDED HOUSING DEVELOPMENT,RCR,314.0,~-,1,618,3,4750000.0,Semi-detached,...,Freehold,NaN,NaN,NaN,1.282130,103.786879,18,"['HILLGROVE SECONDARY SCHOOL', 'QUEENSTOWN SEC...","['Pasir Panjang Food Centre', 'Clementi West S...","['NTUC FAIRPRICE CO-OPERATIVE LTD', 'COLD STOR..."
3,NEO PEE TECK LANE,LANDED HOUSING DEVELOPMENT,RCR,159.3,~-,1,320,3,2630000.0,Terrace,...,Freehold,NaN,NaN,NaN,1.292047,103.768591,20,"['HILLGROVE SECONDARY SCHOOL', 'QUEENSTOWN SEC...","['Pasir Panjang Food Centre', 'Clementi West S...","['NTUC FAIRPRICE CO-OPERATIVE LTD', 'COLD STOR..."
4,COVE DRIVE,TURQUOISE,CCR,223.0,~01-05,1,221,3,2800000.0,Condominium,...,99 yrs lease commencing from 2007,28382.47067,25008.33592,NaN,1.244208,103.827487,21,"['FAIRFIELD METHODIST SCHOOL (SECONDARY)', 'QU...",['Telok Blangah Drive Blk 82 (Telok Blangah Ma...,"['NTUC FAIRPRICE CO-OPERATIVE LTD', 'THE QUALI..."


## 1.2 Combine Combined URA (ameneties_in_district) and Bus Stops

Calculate number of bus stops near each listing

In [12]:
# Formula to calculate distance 
from math import cos, sqrt
R = 6371000 #radius of the Earth in m
def distance(lon1, lat1, lon2, lat2):
    x = (lon2 - lon1) * cos(0.5*(lat2+lat1))
    y = (lat2 - lat1)
    return R * sqrt( x*x + y*y )

In [13]:
busStops = bus_stops.to_dict(orient='records')

In [14]:
# threshold of within 1km
numOfStops = []
busStop_names = [] # to check
for long, lat in zip(ameneties_in_district.longitude, ameneties_in_district.latitude):
    buslist = list(filter(lambda d: distance(d["Longitude"], d["Latitude"], long, lat) <= 1000, busStops))
    busStop_names.append(buslist) # to check
    numOfStops.append(len(buslist))

In [15]:
ameneties_in_district['Bus Stops Nearby'] = numOfStops
ameneties_in_district['Bus Stop Names'] = busStop_names

In [16]:
ameneties_in_district.head()

,street,project,marketSegment,area,floorRange,noOfUnits,contractDate,typeOfSale,price,propertyType,...,y,nettPrice,latitude,longitude,year,school,hawkercentre,supermarkets,Bus Stops Nearby,Bus Stop Names
0,ZEHNDER ROAD,LANDED HOUSING DEVELOPMENT,RCR,524.3,~-,1,220,3,5500000.0,Semi-detached,...,NaN,NaN,1.282130,103.786879,20,"['HILLGROVE SECONDARY SCHOOL', 'QUEENSTOWN SEC...","['Pasir Panjang Food Centre', 'Clementi West S...","['NTUC FAIRPRICE CO-OPERATIVE LTD', 'COLD STOR...",0,[]
1,ZEHNDER ROAD,LANDED HOUSING DEVELOPMENT,RCR,308.0,~-,1,918,3,5000000.0,Semi-detached,...,NaN,NaN,1.282130,103.786879,18,"['HILLGROVE SECONDARY SCHOOL', 'QUEENSTOWN SEC...","['Pasir Panjang Food Centre', 'Clementi West S...","['NTUC FAIRPRICE CO-OPERATIVE LTD', 'COLD STOR...",0,[]
2,ZEHNDER ROAD,LANDED HOUSING DEVELOPMENT,RCR,314.0,~-,1,618,3,4750000.0,Semi-detached,...,NaN,NaN,1.282130,103.786879,18,"['HILLGROVE SECONDARY SCHOOL', 'QUEENSTOWN SEC...","['Pasir Panjang Food Centre', 'Clementi West S...","['NTUC FAIRPRICE CO-OPERATIVE LTD', 'COLD STOR...",0,[]
3,NEO PEE TECK LANE,LANDED HOUSING DEVELOPMENT,RCR,159.3,~-,1,320,3,2630000.0,Terrace,...,NaN,NaN,1.292047,103.768591,20,"['HILLGROVE SECONDARY SCHOOL', 'QUEENSTOWN SEC...","['Pasir Panjang Food Centre', 'Clementi West S...","['NTUC FAIRPRICE CO-OPERATIVE LTD', 'COLD STOR...",2,"[{'BusStopCode': 16081, 'RoadName': 'Pasir Pan..."
4,COVE DRIVE,TURQUOISE,CCR,223.0,~01-05,1,221,3,2800000.0,Condominium,...,25008.33592,NaN,1.244208,103.827487,21,"['FAIRFIELD METHODIST SCHOOL (SECONDARY)', 'QU...",['Telok Blangah Drive Blk 82 (Telok Blangah Ma...,"['NTUC FAIRPRICE CO-OPERATIVE LTD', 'THE QUALI...",0,[]


In [17]:
# check bus stops: Neo Pee Teck Lane
print("longitude: ",ameneties_in_district['longitude'][3])
print("latitude: ",ameneties_in_district['latitude'][3])
print("no. bus stops: ",ameneties_in_district['Bus Stops Nearby'][3])
print("name of bus stops: ",ameneties_in_district['Bus Stop Names'][3])

print()
bus_stops

longitude:  103.7685915
latitude:  1.2920468
no. bus stops:  2
name of bus stops:  [{'BusStopCode': 16081, 'RoadName': 'Pasir Panjang Rd', 'Description': 'Aft Palm Green Condo', 'Latitude': 1.2919577779736298, 'Longitude': 103.76866138900287}, {'BusStopCode': 16089, 'RoadName': 'Pasir Panjang Rd', 'Description': 'Aft Clementi Rd', 'Latitude': 1.2920113834678701, 'Longitude': 103.7690842071612}]



,BusStopCode,RoadName,Description,Latitude,Longitude
0,481,Woodlands Rd,BT PANJANG TEMP BUS PK,1.383764,103.758300
1,1012,Victoria St,Hotel Grand Pacific,1.296848,103.852536
2,1013,Victoria St,St. Joseph's Ch,1.297710,103.853225
3,1019,Victoria St,Bras Basah Cplx,1.296990,103.853022
4,1029,Nth Bridge Rd,Cosmic Insurance Bldg,1.296673,103.854414
...,...,...,...,...,...
5016,99139,Changi Village Rd,Blk 5,1.388195,103.987234
5017,99161,Nicoll Dr,Bef Changi Beach CP 3,1.390262,103.992957
5018,99171,Nicoll Dr,Changi Beach CP 2,1.391128,103.991021
5019,99181,Telok Paku Rd,Bef S'pore Aviation Ac,1.387754,103.988503


In [18]:
# check bus stops: Yishun Central 1
print("longitude: ",ameneties_in_district['longitude'][73170])
print("latitude: ",ameneties_in_district['latitude'][73170])
print("no. bus stops: ",ameneties_in_district['Bus Stops Nearby'][73170])
print("name of bus stops: ",ameneties_in_district['Bus Stop Names'][73170])

print()
bus_stops

longitude:  103.8774636
latitude:  1.3984982
no. bus stops:  1
name of bus stops:  [{'BusStopCode': 67701, 'RoadName': 'Fernvale St', 'Description': 'AFT BLK 467B', 'Latitude': 1.3984935, 'Longitude': 103.877445}]



,BusStopCode,RoadName,Description,Latitude,Longitude
0,481,Woodlands Rd,BT PANJANG TEMP BUS PK,1.383764,103.758300
1,1012,Victoria St,Hotel Grand Pacific,1.296848,103.852536
2,1013,Victoria St,St. Joseph's Ch,1.297710,103.853225
3,1019,Victoria St,Bras Basah Cplx,1.296990,103.853022
4,1029,Nth Bridge Rd,Cosmic Insurance Bldg,1.296673,103.854414
...,...,...,...,...,...
5016,99139,Changi Village Rd,Blk 5,1.388195,103.987234
5017,99161,Nicoll Dr,Bef Changi Beach CP 3,1.390262,103.992957
5018,99171,Nicoll Dr,Changi Beach CP 2,1.391128,103.991021
5019,99181,Telok Paku Rd,Bef S'pore Aviation Ac,1.387754,103.988503


## 1.3 Combine Combined URA (ameneties_in_district) and average overall crime rates 

Average Crime Rates Method
- Using the average crime rates data, find the average overall crime rates for each district
- Average overall crime rates refer to the sum of the average crime rates for all categories

In [19]:
# MAP AVERAGE_CRIMES_BY_LOCATION_V3 [POSTAL] COLUMN INTO A NEW COLUMN NAMED [DISTRICT]

# this dictionary shows the mapping from district > 1st 2 characters of postal
district_postal = { 
  '01': ['01', '02', '03', '04', '05', '06'],
  '02': ['07', '08'],
  '03': ['14', '15', '16'],
  '04': ['09', '10'],
  '05': ['11', '12', '13'], 
  '06': ['17'],
  '07': ['18', '19'],
  '08': ['20', '21'],
  '09': ['22', '23'],
  '10': ['24', '25', '26', '27'],
  '11': ['28', '29', '30'],
  '12': ['31', '32', '33'],
  '13': ['34', '35', '36', '37'],
  '14': ['38', '39', '40', '41'],
  '15': ['42', '43', '44', '45'],
  '16': ['46', '47', '48'],
  '17': ['49', '50', '81'],
  '18': ['51', '52'],
  '19': ['53', '54', '55', '82'],
  '20': ['56', '57'],
  '21': ['58', '59'],
  '22': ['60', '61', '62', '63', '64'],
  '23': ['65', '66', '67', '68'],
  '24': ['69', '70', '71'],
  '25': ['72', '73'],
  '26': ['77', '78'],
  '27': ['75', '76'],
  '28': ['79', '80']
}

# swap dictionary mapping direction
postal_district = {k: oldk for oldk, oldv in district_postal.items() for k in oldv}

# create new column to obtain the 1st 2 characters of [Postal]
average_crimes_by_location_v3['postal prefix'] = average_crimes_by_location_v3['Postal'].astype(str).str[0:2]

# map postal to district code
average_crimes_by_location_v3['district']= average_crimes_by_location_v3['postal prefix'].map(postal_district) 

# get overall crime rate in each district
average_crimes_by_location_v3 = average_crimes_by_location_v3.groupby(['district']).agg({'Number':'sum'}).reset_index()
average_crimes_by_location_v3.head()

,district,Number
0,02,200.000000
1,03,193.285714
2,08,113.000000
3,09,190.714286
4,10,35.428571


In [20]:
# convert str to numpy
district_int = average_crimes_by_location_v3.district.astype(int)
district_int = pd.DataFrame(district_int)
average_crimes_by_location_v3['district'] = district_int

In [21]:
# combine ura_5y_trans and ameneties_per_district
ameneties_and_crime_in_district = ameneties_in_district.join(average_crimes_by_location_v3.set_index('district'), on='district')
print(ameneties_and_crime_in_district.shape)

with pd.option_context('display.max_rows', None, 'display.max_columns', None):  # more options can be specified also
    print(ameneties_and_crime_in_district.head())

(73838, 25)
              street                     project marketSegment   area  \
0       ZEHNDER ROAD  LANDED HOUSING DEVELOPMENT           RCR  524.3   
1       ZEHNDER ROAD  LANDED HOUSING DEVELOPMENT           RCR  308.0   
2       ZEHNDER ROAD  LANDED HOUSING DEVELOPMENT           RCR  314.0   
3  NEO PEE TECK LANE  LANDED HOUSING DEVELOPMENT           RCR  159.3   
4         COVE DRIVE                   TURQUOISE           CCR  223.0   

  floorRange  noOfUnits  contractDate  typeOfSale      price   propertyType  \
0         ~-          1           220           3  5500000.0  Semi-detached   
1         ~-          1           918           3  5000000.0  Semi-detached   
2         ~-          1           618           3  4750000.0  Semi-detached   
3         ~-          1           320           3  2630000.0        Terrace   
4     ~01-05          1           221           3  2800000.0    Condominium   

   district typeOfArea                             tenure            x  \


## 1.4 Compute remaining lease using tenure
- num of years used = now - completion date
- remaining = 99 - num of years used

In [22]:
contractDate_datetime = []
for index in range(len(ameneties_and_crime_in_district)):
    contractdate_text = str(ameneties_and_crime_in_district['contractDate'][index])
    contractdate_month = int(contractdate_text[0:-2])
    contractdate_year = int("20" + contractdate_text[-2:])
    contractdate_datetime = datetime.date(contractdate_year,contractdate_month,1)
    contractDate_datetime.append(contractdate_datetime)

In [23]:
remaining_lease_list = []
for index in range(len(ameneties_and_crime_in_district)):
    tenure_text = str(ameneties_and_crime_in_district['tenure'][index])
    if tenure_text == "Freehold":
        remaining_lease = "Freehold"
    elif tenure_text == "NA":
        remaining_lease = "NA"
    elif tenure_text == "99 years leasehold":
        remaining_lease = "99 years leasehold"
    else:
        tenure_text_split = tenure_text.split()
        num_years = int(tenure_text_split[0])
        last_text = tenure_text_split[len(tenure_text_split)-1]
        
        if last_text == "01/11/2017":
            date_list = last_text.split("/")
            completion_year = int(date_list[2]) 
            completion_month = int(date_list[1])
        else:
            completion_year = int(last_text)
            completion_month = 1 # Assume January
            
        completion_date = datetime.date(completion_year,completion_month,1)
        contract_date = contractDate_datetime[index]
        
        num_months_used = (contract_date.year - completion_date.year) * 12 + (contract_date.month - completion_date.month)
        num_months_available = num_years*12
        
        remaining_lease_months_total = num_months_available-num_months_used
        
        # convert remaining lease in months to years and months
        if remaining_lease_months_total < 12:
            remaining_lease_months = remaining_lease_months_total
            remaining_lease = str(remaining_lease_months) + " months"
        else:
            remaining_lease_years = math.floor(remaining_lease_months_total/12)
            remaining_lease_months = remaining_lease_months_total - remaining_lease_years*12
            remaining_lease = str(remaining_lease_years) + " years " + str(remaining_lease_months) + " months"
            
    remaining_lease_list.append(remaining_lease)

In [24]:
ameneties_and_crime_in_district['remaining_lease'] = remaining_lease_list
ameneties_and_crime_in_district.head()

,street,project,marketSegment,area,floorRange,noOfUnits,contractDate,typeOfSale,price,propertyType,...,latitude,longitude,year,school,hawkercentre,supermarkets,Bus Stops Nearby,Bus Stop Names,Number,remaining_lease
0,ZEHNDER ROAD,LANDED HOUSING DEVELOPMENT,RCR,524.3,~-,1,220,3,5500000.0,Semi-detached,...,1.282130,103.786879,20,"['HILLGROVE SECONDARY SCHOOL', 'QUEENSTOWN SEC...","['Pasir Panjang Food Centre', 'Clementi West S...","['NTUC FAIRPRICE CO-OPERATIVE LTD', 'COLD STOR...",0,[],NaN,Freehold
1,ZEHNDER ROAD,LANDED HOUSING DEVELOPMENT,RCR,308.0,~-,1,918,3,5000000.0,Semi-detached,...,1.282130,103.786879,18,"['HILLGROVE SECONDARY SCHOOL', 'QUEENSTOWN SEC...","['Pasir Panjang Food Centre', 'Clementi West S...","['NTUC FAIRPRICE CO-OPERATIVE LTD', 'COLD STOR...",0,[],NaN,Freehold
2,ZEHNDER ROAD,LANDED HOUSING DEVELOPMENT,RCR,314.0,~-,1,618,3,4750000.0,Semi-detached,...,1.282130,103.786879,18,"['HILLGROVE SECONDARY SCHOOL', 'QUEENSTOWN SEC...","['Pasir Panjang Food Centre', 'Clementi West S...","['NTUC FAIRPRICE CO-OPERATIVE LTD', 'COLD STOR...",0,[],NaN,Freehold
3,NEO PEE TECK LANE,LANDED HOUSING DEVELOPMENT,RCR,159.3,~-,1,320,3,2630000.0,Terrace,...,1.292047,103.768591,20,"['HILLGROVE SECONDARY SCHOOL', 'QUEENSTOWN SEC...","['Pasir Panjang Food Centre', 'Clementi West S...","['NTUC FAIRPRICE CO-OPERATIVE LTD', 'COLD STOR...",2,"[{'BusStopCode': 16081, 'RoadName': 'Pasir Pan...",NaN,Freehold
4,COVE DRIVE,TURQUOISE,CCR,223.0,~01-05,1,221,3,2800000.0,Condominium,...,1.244208,103.827487,21,"['FAIRFIELD METHODIST SCHOOL (SECONDARY)', 'QU...",['Telok Blangah Drive Blk 82 (Telok Blangah Ma...,"['NTUC FAIRPRICE CO-OPERATIVE LTD', 'THE QUALI...",0,[],NaN,84 years 11 months


## 1.5 Additional Cleaning of Columns

In [25]:
# Represent remaining_lease column with numerical values
remaining_lease_numerical = []
for i in range(len(ameneties_and_crime_in_district)):
    text = ameneties_and_crime_in_district['remaining_lease'][i]
    if text == "Freehold":
        num_years = float(999)
    else:
        text_list = text.split(' ')
        len_text_list = len(text_list)
        if 'leasehold' in text:
            num_years = float(text_list[0])
        else:
            years = float(text_list[0])
            months = float(text_list[2])
            num_years_norounding = years + months/12
            num_years = float("{:.2f}".format(num_years_norounding))        
    remaining_lease_numerical.append(num_years)
ameneties_and_crime_in_district['remaining_lease'] = remaining_lease_numerical

In [26]:
ameneties_and_crime_in_district['remaining_lease'].value_counts()

999.00     21371
97.50       1737
97.00       1609
97.33       1323
97.83       1061
           ...  
950.00         1
808.00         1
9943.83        1
50.42          1
975.33         1
Name: remaining_lease, Length: 977, dtype: int64

In [27]:
# Clean up floor range column 
ameneties_and_crime_in_district['floorRange'] = ameneties_and_crime_in_district['floorRange'].map(lambda x: x.lstrip('~'))
# For Listings with > 1 Unit make sure its the average
ameneties_and_crime_in_district['price'] = np.where(ameneties_and_crime_in_district['noOfUnits'] > 1,
                                           ameneties_and_crime_in_district['price'] / ameneties_and_crime_in_district['noOfUnits'],
                                           ameneties_and_crime_in_district['price'])

ameneties_and_crime_in_district['area'] = np.where(ameneties_and_crime_in_district['noOfUnits'] > 1,
                                           ameneties_and_crime_in_district['area'] / ameneties_and_crime_in_district['noOfUnits'],
                                           ameneties_and_crime_in_district['area'])
# Filter only resale and subsale units (2 and 3)
ameneties_and_crime_in_district = ameneties_and_crime_in_district[ameneties_and_crime_in_district['typeOfSale'].isin([2, 3])]
# rename columns
ameneties_and_crime_in_district = ameneties_and_crime_in_district.rename(columns={"Number":"crime_number", "area": "floor_area_sqm", "floorRange": "floor_range"})

## 1.6 Rearrange columns
    1. district
    2. street
    3. propertyType (add [propertyType]=Flat in resale_flat_prices)
    4. remaining_lease 
    5. price (rename resale_flat_prices[resale_price] as [price]])
    6. school
    7. hawkercentre
    8. supermarkets
    9. Bus Stops Nearby
    10. Crime Number
    11. Latitude
    12. Longitude
    13. area
    14. Floor Range
    15. sentiment

In [28]:
# Combine sentiment and dataset
ameneties_and_crime_in_district.year = ameneties_and_crime_in_district.year + 2000

ameneties_and_crime_in_district['disyear'] = ameneties_and_crime_in_district['district'].astype(str) + "_" + ameneties_and_crime_in_district['year'].astype(str)

sentiment['disyear'] = sentiment['district_num'].astype(str) + '_' + sentiment['year'].astype(str)

ameneties_and_crime_in_district['sentiment'] = ameneties_and_crime_in_district['disyear'].map(sentiment.set_index('disyear')['weighted_sentiment'])

In [29]:
ameneties_and_crime_in_district.isnull().any()

street              False
project             False
marketSegment       False
floor_area_sqm      False
floor_range         False
noOfUnits           False
contractDate        False
typeOfSale          False
price               False
propertyType        False
district            False
typeOfArea          False
tenure              False
x                    True
y                    True
nettPrice            True
latitude            False
longitude           False
year                False
school              False
hawkercentre        False
supermarkets        False
Bus Stops Nearby    False
Bus Stop Names      False
crime_number         True
remaining_lease     False
disyear             False
sentiment           False
dtype: bool

In [30]:
# drop irrelevant columns
ameneties_and_crime_in_district.columns

Index(['street', 'project', 'marketSegment', 'floor_area_sqm', 'floor_range',
       'noOfUnits', 'contractDate', 'typeOfSale', 'price', 'propertyType',
       'district', 'typeOfArea', 'tenure', 'x', 'y', 'nettPrice', 'latitude',
       'longitude', 'year', 'school', 'hawkercentre', 'supermarkets',
       'Bus Stops Nearby', 'Bus Stop Names', 'crime_number', 'remaining_lease',
       'disyear', 'sentiment'],
      dtype='object')

In [31]:
ameneties_and_crime_in_district = ameneties_and_crime_in_district[['district','street','propertyType','remaining_lease','price',
                                                                   'school','hawkercentre','supermarkets',
                                                                   'Bus Stops Nearby','crime_number', 'latitude', 'longitude',
                                                                   'floor_area_sqm', 'floor_range', 'sentiment']]
ameneties_and_crime_in_district.head()

,district,street,propertyType,remaining_lease,price,school,hawkercentre,supermarkets,Bus Stops Nearby,crime_number,latitude,longitude,floor_area_sqm,floor_range,sentiment
0,5,ZEHNDER ROAD,Semi-detached,999.00,5500000.0,"['HILLGROVE SECONDARY SCHOOL', 'QUEENSTOWN SEC...","['Pasir Panjang Food Centre', 'Clementi West S...","['NTUC FAIRPRICE CO-OPERATIVE LTD', 'COLD STOR...",0,NaN,1.282130,103.786879,524.3,-,0.129813
1,5,ZEHNDER ROAD,Semi-detached,999.00,5000000.0,"['HILLGROVE SECONDARY SCHOOL', 'QUEENSTOWN SEC...","['Pasir Panjang Food Centre', 'Clementi West S...","['NTUC FAIRPRICE CO-OPERATIVE LTD', 'COLD STOR...",0,NaN,1.282130,103.786879,308.0,-,0.064396
2,5,ZEHNDER ROAD,Semi-detached,999.00,4750000.0,"['HILLGROVE SECONDARY SCHOOL', 'QUEENSTOWN SEC...","['Pasir Panjang Food Centre', 'Clementi West S...","['NTUC FAIRPRICE CO-OPERATIVE LTD', 'COLD STOR...",0,NaN,1.282130,103.786879,314.0,-,0.064396
3,5,NEO PEE TECK LANE,Terrace,999.00,2630000.0,"['HILLGROVE SECONDARY SCHOOL', 'QUEENSTOWN SEC...","['Pasir Panjang Food Centre', 'Clementi West S...","['NTUC FAIRPRICE CO-OPERATIVE LTD', 'COLD STOR...",2,NaN,1.292047,103.768591,159.3,-,0.129813
4,4,COVE DRIVE,Condominium,84.92,2800000.0,"['FAIRFIELD METHODIST SCHOOL (SECONDARY)', 'QU...",['Telok Blangah Drive Blk 82 (Telok Blangah Ma...,"['NTUC FAIRPRICE CO-OPERATIVE LTD', 'THE QUALI...",0,NaN,1.244208,103.827487,223.0,01-05,0.074654


## 2.1 Combine resale_flat_prices and amenities

In [32]:
# this dictionary shows the mapping from district > 1st 2 characters of postal
district_town = {
  '01': ['Raffles Place', 'Cecil', 'Marina', 'People’s Park'],
  '02': ['Anson', 'Tanjong Pagar'],
  '03': ['Queenstown', 'Tiong Bahru'],
  '04': ['Telok Blangah', 'Harbourfront'],
  '05': ['Pasir Panjang', 'Hong Leong Garden', 'Clementi New Town'], 
  '06': ['High Street', 'Beach Road'],
  '07': ['Middle Road', 'Golden Mile'],
  '08': ['Little India'],
  '09': ['Orchard', 'Cairnhill', 'River Valley'],
  '10': ['Ardmore', 'Bukit Timah', 'Holland Road', 'Tanglin'],
  '11': ['Watten Estate', 'Novena', 'Thomson'],
  '12': ['Balestier', 'Toa Payoh','Serangoon'],
  '13': ['Macpherson', 'Braddell'],
  '14': ['Geylang', 'Eunos'],
  '15': ['Katong', 'Joo Chiat', 'Amber Road'],
  '16': ['Bedok', 'Upper East Coast', 'Eastwood', 'Kew Drive'],
  '17': ['Loyang', 'Changi'],
  '18': ['Tampines', 'Pasir Ris'],
  '19': ['Serangoon Garden', 'Hougang', 'Punggol'],
  '20': ['Bishan', 'Ang Mo Kio'],
  '21': ['Upper Bukit Timah', 'Clementi Park', 'Ulu Pandan'],
  '22': ['Jurong'],
  '23': ['Hillview', 'Dairy Farm', 'Bukit Panjang', 'Choa Chu Kang'],
  '24': ['Lim Chu Kang', 'Tengah'],
  '25': ['Kranji', 'Woodgrove'],
  '26': ['Upper Thomson', 'Springleaf'],
  '27': ['Yishun', 'Sembawang'],
  '28': ['Seletar']
}

# convert all values to upper case
district_town = {key: [ele.upper() for ele in district_town[key] ] for key in district_town }

# swap dictionary mapping direction
town_district = {k: oldk for oldk, oldv in district_town.items() for k in oldv}

# map town to district code
resale_flat_prices['town'].str.upper()  # convert and ensure all town string in the data are in upper case 
resale_flat_prices['district'] = resale_flat_prices['town'].map(town_district)

In [33]:
import requests 
import json

def get_postal_onemap(place, district_postal):
    start_code= "https://developers.onemap.sg/commonapi/search?returnGeom=Y&getAddrDetails=Y&pageNum=1&searchVal="+ str("Ang Mo Kio Ave 5")
    s_response = requests.get(start_code)
    s_data = json.loads(s_response.text)
    postal = None
    count = 0
    while postal is None:
        count += 1
        if count == 10:
            postal = None
            break
        for i in range(len(s_data['results'])):
            postal = s_data['results'][i]['POSTAL']
            try:
                postal = int(postal)
            except:
                continue
    for district, sub_dist in district_postal.items():
        if str(postal)[:2] in sub_dist:
            dist = district
    try:
        return dist
    except:
        return -1

In [34]:
# convert nan to -1
resale_flat_prices['district'] = resale_flat_prices['district'].replace(np.nan, -1)
resale_flat_prices.district.unique()

resale_flat_prices['district'] = np.where(resale_flat_prices['district'] == -1,
                                           get_postal_onemap(resale_flat_prices['district'], district_postal),
                                           resale_flat_prices['district'])
# convert str to numpy
resale_district_int = resale_flat_prices.district.astype(int)
resale_district_int = pd.DataFrame(resale_district_int)
resale_flat_prices['district'] = resale_district_int

In [35]:
# Represent remaining_lease column with numerical values
remaining_lease_numerical = []
for i in range(len(resale_flat_prices)):
    text = resale_flat_prices['remaining_lease'][i]
    if text == "Freehold":
        num_years = float('999')
    else:
        text_list = text.split(' ')
        len_text_list = len(text_list)
        if 'leasehold' in text:
            num_years = float(text_list[0])
        else:
            years = float(text_list[0])
            if len(text_list) == 4:
                months = float(text_list[2])
            else:
                months = 0
            num_years_norounding = years + months/12
            num_years = float("{:.2f}".format(num_years_norounding))        
    remaining_lease_numerical.append(num_years)
resale_flat_prices['remaining_lease'] = remaining_lease_numerical

In [36]:
# combine resale_flat_prices and ameneties_per_district
resale_flats_in_district = resale_flat_prices.join(ameneties_per_district.set_index('district'), on='district')
print(resale_flats_in_district.shape)
resale_flats_in_district.head()

(92235, 18)


,month,town,flat_type,block,street_name,storey_range,floor_area_sqm,flat_model,lease_commence_date,remaining_lease,resale_price,street,latitude,longitude,district,school,hawkercentre,supermarkets
0,2017-01,ANG MO KIO,2 ROOM,406,ANG MO KIO AVE 10,10 TO 12,44.0,Improved,1979,61.33,232000.0,ANG MO KIO AVE 10,1.367602,103.856844,20,"['BEATTY SECONDARY SCHOOL', 'MAYFLOWER SECONDA...","['Ang Mo Kio Ave 1 Blk 341 (Teck Ghee Court)',...","['NTUC FAIRPRICE CO-OPERATIVE LTD', 'G8 MART P..."
1,2017-01,ANG MO KIO,3 ROOM,108,ANG MO KIO AVE 4,01 TO 03,67.0,New Generation,1978,60.58,250000.0,ANG MO KIO AVE 4,1.382319,103.838706,20,"['BEATTY SECONDARY SCHOOL', 'MAYFLOWER SECONDA...","['Ang Mo Kio Ave 1 Blk 341 (Teck Ghee Court)',...","['NTUC FAIRPRICE CO-OPERATIVE LTD', 'G8 MART P..."
2,2017-01,ANG MO KIO,3 ROOM,602,ANG MO KIO AVE 5,01 TO 03,67.0,New Generation,1980,62.42,262000.0,ANG MO KIO AVE 5,1.376825,103.847556,20,"['BEATTY SECONDARY SCHOOL', 'MAYFLOWER SECONDA...","['Ang Mo Kio Ave 1 Blk 341 (Teck Ghee Court)',...","['NTUC FAIRPRICE CO-OPERATIVE LTD', 'G8 MART P..."
3,2017-01,ANG MO KIO,3 ROOM,465,ANG MO KIO AVE 10,04 TO 06,68.0,New Generation,1980,62.08,265000.0,ANG MO KIO AVE 10,1.367602,103.856844,20,"['BEATTY SECONDARY SCHOOL', 'MAYFLOWER SECONDA...","['Ang Mo Kio Ave 1 Blk 341 (Teck Ghee Court)',...","['NTUC FAIRPRICE CO-OPERATIVE LTD', 'G8 MART P..."
4,2017-01,ANG MO KIO,3 ROOM,601,ANG MO KIO AVE 5,01 TO 03,67.0,New Generation,1980,62.42,265000.0,ANG MO KIO AVE 5,1.376825,103.847556,20,"['BEATTY SECONDARY SCHOOL', 'MAYFLOWER SECONDA...","['Ang Mo Kio Ave 1 Blk 341 (Teck Ghee Court)',...","['NTUC FAIRPRICE CO-OPERATIVE LTD', 'G8 MART P..."


## 2.2 Combine Combined resale_flat_prices (resale_flats_in_district) and Bus Stops

In [37]:
# threshold of within 1km
numOfStops = []
for long, lat in zip(resale_flats_in_district.longitude, resale_flats_in_district.latitude):
    buslist = list(filter(lambda d: distance(d["Longitude"], d["Latitude"], long, lat) <= 1000, busStops))
    numOfStops.append(len(buslist))

resale_flats_in_district['Bus Stops Nearby'] = numOfStops
resale_flats_in_district.head()

,month,town,flat_type,block,street_name,storey_range,floor_area_sqm,flat_model,lease_commence_date,remaining_lease,resale_price,street,latitude,longitude,district,school,hawkercentre,supermarkets,Bus Stops Nearby
0,2017-01,ANG MO KIO,2 ROOM,406,ANG MO KIO AVE 10,10 TO 12,44.0,Improved,1979,61.33,232000.0,ANG MO KIO AVE 10,1.367602,103.856844,20,"['BEATTY SECONDARY SCHOOL', 'MAYFLOWER SECONDA...","['Ang Mo Kio Ave 1 Blk 341 (Teck Ghee Court)',...","['NTUC FAIRPRICE CO-OPERATIVE LTD', 'G8 MART P...",0
1,2017-01,ANG MO KIO,3 ROOM,108,ANG MO KIO AVE 4,01 TO 03,67.0,New Generation,1978,60.58,250000.0,ANG MO KIO AVE 4,1.382319,103.838706,20,"['BEATTY SECONDARY SCHOOL', 'MAYFLOWER SECONDA...","['Ang Mo Kio Ave 1 Blk 341 (Teck Ghee Court)',...","['NTUC FAIRPRICE CO-OPERATIVE LTD', 'G8 MART P...",0
2,2017-01,ANG MO KIO,3 ROOM,602,ANG MO KIO AVE 5,01 TO 03,67.0,New Generation,1980,62.42,262000.0,ANG MO KIO AVE 5,1.376825,103.847556,20,"['BEATTY SECONDARY SCHOOL', 'MAYFLOWER SECONDA...","['Ang Mo Kio Ave 1 Blk 341 (Teck Ghee Court)',...","['NTUC FAIRPRICE CO-OPERATIVE LTD', 'G8 MART P...",0
3,2017-01,ANG MO KIO,3 ROOM,465,ANG MO KIO AVE 10,04 TO 06,68.0,New Generation,1980,62.08,265000.0,ANG MO KIO AVE 10,1.367602,103.856844,20,"['BEATTY SECONDARY SCHOOL', 'MAYFLOWER SECONDA...","['Ang Mo Kio Ave 1 Blk 341 (Teck Ghee Court)',...","['NTUC FAIRPRICE CO-OPERATIVE LTD', 'G8 MART P...",0
4,2017-01,ANG MO KIO,3 ROOM,601,ANG MO KIO AVE 5,01 TO 03,67.0,New Generation,1980,62.42,265000.0,ANG MO KIO AVE 5,1.376825,103.847556,20,"['BEATTY SECONDARY SCHOOL', 'MAYFLOWER SECONDA...","['Ang Mo Kio Ave 1 Blk 341 (Teck Ghee Court)',...","['NTUC FAIRPRICE CO-OPERATIVE LTD', 'G8 MART P...",0


## 2.3 Combine Combined resale_flat_prices (resale_flats_in_district) and average overall crime rates

In [38]:
# combine ura_5y_trans and ameneties_per_district
resale_flats_and_crime_in_district = resale_flats_in_district.join(average_crimes_by_location_v3.set_index('district'), on='district')
print(resale_flats_and_crime_in_district.shape)
resale_flats_and_crime_in_district.head()

(92235, 20)


,month,town,flat_type,block,street_name,storey_range,floor_area_sqm,flat_model,lease_commence_date,remaining_lease,resale_price,street,latitude,longitude,district,school,hawkercentre,supermarkets,Bus Stops Nearby,Number
0,2017-01,ANG MO KIO,2 ROOM,406,ANG MO KIO AVE 10,10 TO 12,44.0,Improved,1979,61.33,232000.0,ANG MO KIO AVE 10,1.367602,103.856844,20,"['BEATTY SECONDARY SCHOOL', 'MAYFLOWER SECONDA...","['Ang Mo Kio Ave 1 Blk 341 (Teck Ghee Court)',...","['NTUC FAIRPRICE CO-OPERATIVE LTD', 'G8 MART P...",0,122.714286
1,2017-01,ANG MO KIO,3 ROOM,108,ANG MO KIO AVE 4,01 TO 03,67.0,New Generation,1978,60.58,250000.0,ANG MO KIO AVE 4,1.382319,103.838706,20,"['BEATTY SECONDARY SCHOOL', 'MAYFLOWER SECONDA...","['Ang Mo Kio Ave 1 Blk 341 (Teck Ghee Court)',...","['NTUC FAIRPRICE CO-OPERATIVE LTD', 'G8 MART P...",0,122.714286
2,2017-01,ANG MO KIO,3 ROOM,602,ANG MO KIO AVE 5,01 TO 03,67.0,New Generation,1980,62.42,262000.0,ANG MO KIO AVE 5,1.376825,103.847556,20,"['BEATTY SECONDARY SCHOOL', 'MAYFLOWER SECONDA...","['Ang Mo Kio Ave 1 Blk 341 (Teck Ghee Court)',...","['NTUC FAIRPRICE CO-OPERATIVE LTD', 'G8 MART P...",0,122.714286
3,2017-01,ANG MO KIO,3 ROOM,465,ANG MO KIO AVE 10,04 TO 06,68.0,New Generation,1980,62.08,265000.0,ANG MO KIO AVE 10,1.367602,103.856844,20,"['BEATTY SECONDARY SCHOOL', 'MAYFLOWER SECONDA...","['Ang Mo Kio Ave 1 Blk 341 (Teck Ghee Court)',...","['NTUC FAIRPRICE CO-OPERATIVE LTD', 'G8 MART P...",0,122.714286
4,2017-01,ANG MO KIO,3 ROOM,601,ANG MO KIO AVE 5,01 TO 03,67.0,New Generation,1980,62.42,265000.0,ANG MO KIO AVE 5,1.376825,103.847556,20,"['BEATTY SECONDARY SCHOOL', 'MAYFLOWER SECONDA...","['Ang Mo Kio Ave 1 Blk 341 (Teck Ghee Court)',...","['NTUC FAIRPRICE CO-OPERATIVE LTD', 'G8 MART P...",0,122.714286


## 2.4 Rearrange columns
    1. district
    2. street
    3. propertyType (add [propertyType]=flat_type in resale_flat_prices)
    4. remaining_lease 
    5. price (rename resale_flat_prices[resale_price] as [price]])
    6. school
    7. hawkercentre
    8. supermarkets
    9. Bus Stops Nearby
    10. Crime Number
    11. sentiment

In [39]:
def extract_year(x):
    splitted = x.split('-')
    return splitted[0]

resale_flats_and_crime_in_district['year'] = resale_flats_and_crime_in_district.month.apply(extract_year)

resale_flats_and_crime_in_district['disyear'] = resale_flats_and_crime_in_district['district'].astype(str) + '_' + resale_flats_and_crime_in_district['year'].astype(str)

In [40]:
avg_score = ['10_2017', '12_2017', '14_2017', '16_2017', '18_2017', '19_2017','20_2017',
 '23_2017', '27_2017', '3_2017']

In [41]:
for value in avg_score:
    if value == '3_2017':
        curr_district = value[0]
    else:
        curr_district = value[0:2]
    
    next_value = curr_district + '_2018'
    curr_sentiment = sentiment[sentiment.disyear == next_value]['weighted_sentiment'].item()
    
    sentiment = sentiment.append({'district_num': curr_district, 'disyear': value,
                                 'hwz_comment_count': None,'hwz_sentiment': None,
                                 'reddit_comment_count': None, 'reddit_sentiment': None,
                                 'total_count': None, 'weighted_sentiment': curr_sentiment,
                                 'year': 2017}, ignore_index=True)

In [42]:
resale_flats_and_crime_in_district['sentiment'] = resale_flats_and_crime_in_district['disyear'].map(sentiment.set_index('disyear')['weighted_sentiment'])

In [43]:
resale_flats_and_crime_in_district.isnull().any()

month                  False
town                   False
flat_type              False
block                  False
street_name            False
storey_range           False
floor_area_sqm         False
flat_model             False
lease_commence_date    False
remaining_lease        False
resale_price           False
street                 False
latitude               False
longitude              False
district               False
school                 False
hawkercentre           False
supermarkets           False
Bus Stops Nearby       False
Number                 False
year                   False
disyear                False
sentiment              False
dtype: bool

In [44]:
# drop irrelevant columns
resale_flats_and_crime_in_district = resale_flats_and_crime_in_district.drop(['month', 'town','block', 'street_name', 'flat_model', 'lease_commence_date'], axis = 1) 
resale_flats_and_crime_in_district.columns

Index(['flat_type', 'storey_range', 'floor_area_sqm', 'remaining_lease',
       'resale_price', 'street', 'latitude', 'longitude', 'district', 'school',
       'hawkercentre', 'supermarkets', 'Bus Stops Nearby', 'Number', 'year',
       'disyear', 'sentiment'],
      dtype='object')

In [45]:
# add property type
resale_flats_and_crime_in_district['propertyType'] = resale_flats_and_crime_in_district['flat_type']
resale_flats_and_crime_in_district = resale_flats_and_crime_in_district.drop(['flat_type'], axis = 1)

# rename resale price as price, Number as crime_number
resale_flats_and_crime_in_district = resale_flats_and_crime_in_district.rename(columns={"resale_price": "price","Number":"crime_number", "storey_range": "floor_range"})

In [46]:
resale_flats_and_crime_in_district = resale_flats_and_crime_in_district[['district','street','propertyType','remaining_lease','price',
                                                                         'school','hawkercentre','supermarkets',
                                                                         'Bus Stops Nearby','crime_number', 'latitude',
                                                                         'longitude', 'floor_area_sqm', 'floor_range', 'sentiment']]
resale_flats_and_crime_in_district.head()

,district,street,propertyType,remaining_lease,price,school,hawkercentre,supermarkets,Bus Stops Nearby,crime_number,latitude,longitude,floor_area_sqm,floor_range,sentiment
0,20,ANG MO KIO AVE 10,2 ROOM,61.33,232000.0,"['BEATTY SECONDARY SCHOOL', 'MAYFLOWER SECONDA...","['Ang Mo Kio Ave 1 Blk 341 (Teck Ghee Court)',...","['NTUC FAIRPRICE CO-OPERATIVE LTD', 'G8 MART P...",0,122.714286,1.367602,103.856844,44.0,10 TO 12,0.056981
1,20,ANG MO KIO AVE 4,3 ROOM,60.58,250000.0,"['BEATTY SECONDARY SCHOOL', 'MAYFLOWER SECONDA...","['Ang Mo Kio Ave 1 Blk 341 (Teck Ghee Court)',...","['NTUC FAIRPRICE CO-OPERATIVE LTD', 'G8 MART P...",0,122.714286,1.382319,103.838706,67.0,01 TO 03,0.056981
2,20,ANG MO KIO AVE 5,3 ROOM,62.42,262000.0,"['BEATTY SECONDARY SCHOOL', 'MAYFLOWER SECONDA...","['Ang Mo Kio Ave 1 Blk 341 (Teck Ghee Court)',...","['NTUC FAIRPRICE CO-OPERATIVE LTD', 'G8 MART P...",0,122.714286,1.376825,103.847556,67.0,01 TO 03,0.056981
3,20,ANG MO KIO AVE 10,3 ROOM,62.08,265000.0,"['BEATTY SECONDARY SCHOOL', 'MAYFLOWER SECONDA...","['Ang Mo Kio Ave 1 Blk 341 (Teck Ghee Court)',...","['NTUC FAIRPRICE CO-OPERATIVE LTD', 'G8 MART P...",0,122.714286,1.367602,103.856844,68.0,04 TO 06,0.056981
4,20,ANG MO KIO AVE 5,3 ROOM,62.42,265000.0,"['BEATTY SECONDARY SCHOOL', 'MAYFLOWER SECONDA...","['Ang Mo Kio Ave 1 Blk 341 (Teck Ghee Court)',...","['NTUC FAIRPRICE CO-OPERATIVE LTD', 'G8 MART P...",0,122.714286,1.376825,103.847556,67.0,01 TO 03,0.056981


In [47]:
replace_floors = {
    '01 TO 03' : '01-05',
    '04 TO 06' : '01-05',
    '07 TO 09' : '06-10',
    '10 TO 12' : '11-15',
    '13 TO 15' : '11-15',
    '16 TO 18' : '16-20',
    '19 TO 21' : '16-20',
    '22 TO 24' : '21-25',
    '25 TO 27' : '26-30',
    '28 TO 30' : '26-30',
    '31 TO 33' : '31-35',
    '34 TO 36' : '31-35',
    '37 TO 39' : '36-40',
    '40 TO 42' : '41-45',
    '43 TO 45' : '41-45',
    '46 TO 48' : '46-50',
    
}

resale_flats_and_crime_in_district = resale_flats_and_crime_in_district.replace({'floor_range': replace_floors})

## 3. Combine ameneties_and_crime_in_district and resale_flats_and_crime_in_district

In [48]:
df = ameneties_and_crime_in_district.append(resale_flats_and_crime_in_district)
df.head()

,district,street,propertyType,remaining_lease,price,school,hawkercentre,supermarkets,Bus Stops Nearby,crime_number,latitude,longitude,floor_area_sqm,floor_range,sentiment
0,5,ZEHNDER ROAD,Semi-detached,999.00,5500000.0,"['HILLGROVE SECONDARY SCHOOL', 'QUEENSTOWN SEC...","['Pasir Panjang Food Centre', 'Clementi West S...","['NTUC FAIRPRICE CO-OPERATIVE LTD', 'COLD STOR...",0,NaN,1.282130,103.786879,524.3,-,0.129813
1,5,ZEHNDER ROAD,Semi-detached,999.00,5000000.0,"['HILLGROVE SECONDARY SCHOOL', 'QUEENSTOWN SEC...","['Pasir Panjang Food Centre', 'Clementi West S...","['NTUC FAIRPRICE CO-OPERATIVE LTD', 'COLD STOR...",0,NaN,1.282130,103.786879,308.0,-,0.064396
2,5,ZEHNDER ROAD,Semi-detached,999.00,4750000.0,"['HILLGROVE SECONDARY SCHOOL', 'QUEENSTOWN SEC...","['Pasir Panjang Food Centre', 'Clementi West S...","['NTUC FAIRPRICE CO-OPERATIVE LTD', 'COLD STOR...",0,NaN,1.282130,103.786879,314.0,-,0.064396
3,5,NEO PEE TECK LANE,Terrace,999.00,2630000.0,"['HILLGROVE SECONDARY SCHOOL', 'QUEENSTOWN SEC...","['Pasir Panjang Food Centre', 'Clementi West S...","['NTUC FAIRPRICE CO-OPERATIVE LTD', 'COLD STOR...",2,NaN,1.292047,103.768591,159.3,-,0.129813
4,4,COVE DRIVE,Condominium,84.92,2800000.0,"['FAIRFIELD METHODIST SCHOOL (SECONDARY)', 'QU...",['Telok Blangah Drive Blk 82 (Telok Blangah Ma...,"['NTUC FAIRPRICE CO-OPERATIVE LTD', 'THE QUALI...",0,NaN,1.244208,103.827487,223.0,01-05,0.074654


In [49]:
df.shape

(132607, 15)

In [50]:
df.columns

Index(['district', 'street', 'propertyType', 'remaining_lease', 'price',
       'school', 'hawkercentre', 'supermarkets', 'Bus Stops Nearby',
       'crime_number', 'latitude', 'longitude', 'floor_area_sqm',
       'floor_range', 'sentiment'],
      dtype='object')

In [51]:
df.isnull().any()

district            False
street              False
propertyType        False
remaining_lease     False
price               False
school              False
hawkercentre        False
supermarkets        False
Bus Stops Nearby    False
crime_number         True
latitude            False
longitude           False
floor_area_sqm      False
floor_range         False
sentiment           False
dtype: bool

## 4. Further cleaning of data

In [52]:
# convert nan to -1
df['crime_number'] = df['crime_number'].replace(np.nan, -1)
df.crime_number.unique()

array([ -1.        , 193.28571429, 200.        ,  35.42857143,
       113.        , 190.71428571, 167.71428571,  60.85714286,
        45.42857143, 202.14285714, 130.        , 122.71428571,
       163.85714286, 169.42857143, 226.        ,  64.        ,
        78.92857143])

In [53]:
df.isnull().any()

district            False
street              False
propertyType        False
remaining_lease     False
price               False
school              False
hawkercentre        False
supermarkets        False
Bus Stops Nearby    False
crime_number        False
latitude            False
longitude           False
floor_area_sqm      False
floor_range         False
sentiment           False
dtype: bool

In [54]:
# Convert list of schools, hawkers and supermarkets into num of each facility around
# School

import json

num_sch = []
for sch in df['school']:
    curr = sch.strip('][').split(',') 
    if " SINGAPORE'" in curr:
        curr.remove(" SINGAPORE'")
    num_sch.append(len(curr))

df['school'] = num_sch

# Hawker Centre
num_hawker = []
for hawker in df['hawkercentre']:
    curr = hawker.strip('][').split(',')
    num_hawker.append(len(curr))

df['hawkercentre'] = num_hawker

# Supermarkets
num_supermarkets = []
for supermarket in df['supermarkets']:
    curr = supermarket.strip('][').split(',')
    num_supermarkets.append(len(curr))

df['supermarkets'] = num_supermarkets

df

,district,street,propertyType,remaining_lease,price,school,hawkercentre,supermarkets,Bus Stops Nearby,crime_number,latitude,longitude,floor_area_sqm,floor_range,sentiment
0,5,ZEHNDER ROAD,Semi-detached,999.00,5500000.0,27,6,8,0,-1.0,1.282130,103.786879,524.3,-,0.129813
1,5,ZEHNDER ROAD,Semi-detached,999.00,5000000.0,27,6,8,0,-1.0,1.282130,103.786879,308.0,-,0.064396
2,5,ZEHNDER ROAD,Semi-detached,999.00,4750000.0,27,6,8,0,-1.0,1.282130,103.786879,314.0,-,0.064396
3,5,NEO PEE TECK LANE,Terrace,999.00,2630000.0,27,6,8,2,-1.0,1.292047,103.768591,159.3,-,0.129813
4,4,COVE DRIVE,Condominium,84.92,2800000.0,11,3,2,0,-1.0,1.244208,103.827487,223.0,01-05,0.074654
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
92230,27,YISHUN AVE 6,EXECUTIVE,66.67,655000.0,18,2,19,0,64.0,1.426142,103.849653,148.0,11-15,0.037938
92231,27,YISHUN RING ROAD,EXECUTIVE,66.83,650000.0,18,2,19,0,64.0,1.424340,103.845070,146.0,11-15,0.037938
92232,27,YISHUN RING ROAD,EXECUTIVE,66.17,607000.0,18,2,19,0,64.0,1.424340,103.845070,146.0,01-05,0.037938
92233,27,YISHUN ST 81,EXECUTIVE,66.00,670000.0,18,2,19,0,64.0,1.415253,103.837021,142.0,01-05,0.037938


In [55]:
df.isnull().any()

district            False
street              False
propertyType        False
remaining_lease     False
price               False
school              False
hawkercentre        False
supermarkets        False
Bus Stops Nearby    False
crime_number        False
latitude            False
longitude           False
floor_area_sqm      False
floor_range         False
sentiment           False
dtype: bool

## 5. Normalise Data and Split Data

In [56]:
# Label Encoding 
df['street'] = df['street'].astype('category')
df['street'] = df['street'].cat.codes

df['propertyType'] = df['propertyType'].astype('category')
df['propertyType'] = df['propertyType'].cat.codes

df['floor_range'] = df['floor_range'].astype('category')
df['floor_range'] = df['floor_range'].cat.codes

In [57]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(df.drop('price', axis=1),
                                                    df['price'], 
                                                    test_size=0.1,
                                                    random_state=1)
print(X_train.shape)
print(X_test.shape)
print(y_train.shape)
print(y_test.shape)

(119346, 14)
(13261, 14)
(119346,)
(13261,)


In [58]:
# Normalise remaining lease for training and test
X_train_lease_mean = X_train['remaining_lease'].mean()

X_train_lease_std = X_train['remaining_lease'].std()
X_train['remaining_lease'] = (X_train['remaining_lease'] - X_train_lease_mean)/X_train_lease_std

X_test['remaining_lease'] = (X_test['remaining_lease'] - X_train_lease_mean)/X_train_lease_std


# Normalise floor area for training and test
X_train_area_mean = X_train['floor_area_sqm'].mean()

X_train_area_std = X_train['floor_area_sqm'].std()
X_train['floor_area_sqm'] = (X_train['floor_area_sqm'] - X_train_area_mean)/X_train_area_std

X_test['floor_area_sqm'] = (X_test['floor_area_sqm'] - X_train_area_mean)/X_train_area_std

In [59]:
y_train = pd.Series(y_train,
              name="price").to_frame()

y_test = pd.Series(y_test,
              name="price").to_frame()

In [60]:
# Normalise price for training and test
y_train_price_mean = y_train['price'].mean()

y_train_price_std = y_train['price'].std()
y_train['price'] = (y_train['price'] - y_train_price_mean)/y_train_price_std

y_test['price'] = (y_test['price'] - y_train_price_mean)/y_train_price_std

In [61]:
print(X_train.shape)
print(X_train.isnull().any())
print()
print(X_test.shape)
print(X_test.isnull().any())
print()
print(y_train.shape)
print(y_train.isnull().any())
print()
print(y_test.shape)
print(y_test.isnull().any())
print()

(119346, 14)
district            False
street              False
propertyType        False
remaining_lease     False
school              False
hawkercentre        False
supermarkets        False
Bus Stops Nearby    False
crime_number        False
latitude            False
longitude           False
floor_area_sqm      False
floor_range         False
sentiment           False
dtype: bool

(13261, 14)
district            False
street              False
propertyType        False
remaining_lease     False
school              False
hawkercentre        False
supermarkets        False
Bus Stops Nearby    False
crime_number        False
latitude            False
longitude           False
floor_area_sqm      False
floor_range         False
sentiment           False
dtype: bool

(119346, 1)
price    False
dtype: bool

(13261, 1)
price    False
dtype: bool



## 6. Export to csv

In [65]:
# X_train.to_csv("/Users/vickiyew/Documents/BT4222/project/data/training_set.csv", index = False)
# X_test.to_csv("/Users/vickiyew/Documents/BT4222/project/data/testing_set.csv", index = False)
# y_train.to_csv("/Users/vickiyew/Documents/BT4222/project/data/training_labels.csv", index = False)
# y_test.to_csv("/Users/vickiyew/Documents/BT4222/project/data/testing_labels.csv", index = False)
# df.to_csv("/Users/vickiyew/Documents/BT4222/project/data/combined.csv", index = False)

In [63]:
#df.to_csv("/Users/admin/Desktop/Y3S2/BT4222/combined.csv", index = False)

In [64]:
#df.to_csv('combined.csv')